[View in Colaboratory](https://colab.research.google.com/github/McCloud77/DeepLearning/blob/master/Deep_Learning_9_Data_Augmentation.ipynb)

## Classifying CIFAR-10 with Data Augmentation.

Here we revisit CIFAR-10 and the networks we previously built.  We will use real-time data augmentation to try to improve our results.

In [2]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [3]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

170500096/170498071 [==============================] - 10s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [0]:
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

First, let's use a smaller model with 181,162 prameters and train it with data augmentation.

In [6]:
# CNN using Keras' Sequential capabilities
model_1 = Sequential()

## 5x5 convolution with 2x2 stride and 32 filters
model_1.add(Conv2D(32, (5, 5),
                   strides = (2,2),
                   padding='same',
                   input_shape=x_train.shape[1:]))
model_1.add(Activation('relu'))

## Another 5x5 convolution with 2x2 stride and 32 filters
model_1.add(Conv2D(32, (5, 5),
                   strides = (2,2)))
model_1.add(Activation('relu'))

## 2x2 max pooling reduces to 3 x 3 x 32
model_1.add(MaxPooling2D(pool_size=(2, 2)))
model_1.add(Dropout(0.25))

## Flatten turns 3x3x32 into 288x1
model_1.add(Flatten())
model_1.add(Dense(512))
model_1.add(Activation('relu'))
model_1.add(Dropout(0.5))
model_1.add(Dense(num_classes))
model_1.add(Activation('softmax'))

model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 32)        2432      
_________________________________________________________________
activation_1 (Activation)    (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 32)          25632     
_________________________________________________________________
activation_2 (Activation)    (None, 6, 6, 32)          0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 3, 3, 32)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 3, 3, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 288)               0         
__________

In [0]:
batch_size = 32

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0005, decay=1e-6)

# Let's train the model using RMSprop
model_1.compile(loss='categorical_crossentropy',
                optimizer=opt,
                metrics=['accuracy'])

Here we define the `ImageDataGenerator` that we will use to serve images to our model during the training process.  Currently, it is configured to do some shifting and horizontal flipping.

In [8]:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)   # randomly flip images
                   
datagen.fit(x_train)      # This computes any statistics that may be needed (e.g. for centering) from the training set.
    
# Fit the model on the batches generated by datagen.flow().
model_1.fit_generator(datagen.flow(x_train, y_train,
                                   batch_size=batch_size),
                      steps_per_epoch=x_train.shape[0] // batch_size,
                      epochs=15,
                      verbose=1,
                      validation_data=(x_test, y_test))

Epoch 1/15
1562/1562 [==============================] - 42s 27ms/step - loss: 1.7863 - acc: 0.3455 - val_loss: 1.5897 - val_acc: 0.4428
Epoch 2/15
1562/1562 [==============================] - 41s 26ms/step - loss: 1.5317 - acc: 0.4459 - val_loss: 1.3375 - val_acc: 0.5224
Epoch 3/15
1562/1562 [==============================] - 41s 26ms/step - loss: 1.4402 - acc: 0.4833 - val_loss: 1.2635 - val_acc: 0.5498
Epoch 4/15
1562/1562 [==============================] - 40s 26ms/step - loss: 1.3868 - acc: 0.5017 - val_loss: 1.2090 - val_acc: 0.5698
Epoch 5/15
1562/1562 [==============================] - 40s 25ms/step - loss: 1.3475 - acc: 0.5172 - val_loss: 1.2300 - val_acc: 0.5656
Epoch 6/15
1562/1562 [==============================] - 40s 25ms/step - loss: 1.3306 - acc: 0.5257 - val_loss: 1.1787 - val_acc: 0.5823
Epoch 7/15
1562/1562 [==============================] - 40s 26ms/step - loss: 1.3093 - acc: 0.5365 - val_loss: 1.1383 - val_acc: 0.5987
Epoch 8/15
1562/1562 [==========================

In [9]:
# Testing
scores = model_1.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

10000/10000 [==============================] - 0s 46us/step

Test result: 60.560 loss: 1.152


## Using Regularization and Different Learning Rates.

In [0]:
from keras.utils import np_utils
from keras.layers import BatchNormalization
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np

In [0]:
# Change learning rate based on epoch
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    elif epoch > 100:
        lrate = 0.0003
    return lrate

In [0]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Calculate Z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

In [0]:
weight_decay = 1e-4

In [14]:
model_2 = Sequential()

model_2.add(Conv2D(32, (3,3), padding='same',
                   kernel_regularizer=regularizers.l2(weight_decay),
                   input_shape=x_train.shape[1:]))
model_2.add(BatchNormalization())
model_2.add(Activation('relu'))
      
      
model_2.add(Conv2D(32, (3,3), padding='same',
                   kernel_regularizer=regularizers.l2(weight_decay)))
model_2.add(BatchNormalization())
model_2.add(Activation('relu'))
model_2.add(MaxPooling2D(pool_size=(2,2)))
model_2.add(Dropout(0.2))

model_2.add(Conv2D(64, (3,3), padding='same',
                 kernel_regularizer=regularizers.l2(weight_decay)))
model_2.add(BatchNormalization())
model_2.add(Activation('relu'))
            
model_2.add(Conv2D(64, (3,3), padding='same',
                 kernel_regularizer=regularizers.l2(weight_decay)))
model_2.add(BatchNormalization())
model_2.add(Activation('relu'))
model_2.add(MaxPooling2D(pool_size=(2,2)))
model_2.add(Dropout(0.3))
            
model_2.add(Conv2D(128, (3,3), padding='same',
                   kernel_regularizer=regularizers.l2(weight_decay)))
model_2.add(BatchNormalization())
model_2.add(Activation('relu'))
model_2.add(Conv2D(128, (3,3), padding='same',
                   kernel_regularizer=regularizers.l2(weight_decay)))
model_2.add(BatchNormalization())
model_2.add(Activation('relu'))
model_2.add(MaxPooling2D(pool_size=(2,2)))
model_2.add(Dropout(0.4))

model_2.add(Flatten())
model_2.add(Dense(num_classes, activation='softmax'))

model_2.summary()            

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
activation_5 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
activation_6 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 32)        0         
__________

In [0]:
# Data Augmentation
datagen = ImageDataGenerator(rotation_range=15,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             horizontal_flip=True,)
datagen.fit(x_train)

In [16]:
batch_size = 32

opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)

model_2.compile(loss='categorical_crossentropy',
              optimizer=opt_rms, metrics=['accuracy'])

model_2.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=125,\
                    verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)])

Epoch 1/125
1562/1562 [==============================] - 68s 44ms/step - loss: 1.6221 - acc: 0.4514 - val_loss: 1.5179 - val_acc: 0.5230
Epoch 2/125
1562/1562 [==============================] - 68s 44ms/step - loss: 1.1759 - acc: 0.6079 - val_loss: 1.0990 - val_acc: 0.6460
Epoch 3/125
1562/1562 [==============================] - 68s 43ms/step - loss: 1.0508 - acc: 0.6653 - val_loss: 0.9052 - val_acc: 0.7149
Epoch 4/125
1562/1562 [==============================] - 68s 43ms/step - loss: 0.9853 - acc: 0.6932 - val_loss: 0.9961 - val_acc: 0.7000
Epoch 5/125
1562/1562 [==============================] - 69s 44ms/step - loss: 0.9465 - acc: 0.7111 - val_loss: 0.9200 - val_acc: 0.7314
Epoch 6/125
1562/1562 [==============================] - 69s 44ms/step - loss: 0.9102 - acc: 0.7255 - val_loss: 0.9043 - val_acc: 0.7441
Epoch 7/125
1562/1562 [==============================] - 68s 43ms/step - loss: 0.8982 - acc: 0.7357 - val_loss: 0.8815 - val_acc: 0.7446
Epoch 8/125
1562/1562 [==================

In [18]:
# Testing
scores = model_2.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

10000/10000 [==============================] - 2s 195us/step

Test result: 83.720 loss: 0.668
